Day 7: Handy Haversacks
You land at the regional airport in time for your next flight. In fact, it looks like you'll even have time to grab some food: all flights are currently delayed due to issues in luggage processing.

Due to recent aviation regulations, many rules (your puzzle input) are being enforced about bags and their contents; bags must be color-coded and must contain specific quantities of other color-coded bags. Apparently, nobody responsible for these regulations considered how long they would take to enforce!

For example, consider the following rules:

```
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
```
These rules specify the required contents for 9 bag types. In this example, every faded blue bag is empty, every vibrant plum bag contains 
11 bags (5 faded blue and 6 dotted black), and so on.

You have a shiny gold bag. If you wanted to carry it in at least one other bag, how many different bag colors would be valid for the outermost bag? 
(In other words: how many colors can, eventually, contain at least one shiny gold bag?)

In the above rules, the following options would be available to you:

A bright white bag, which can hold your shiny gold bag directly.
A muted yellow bag, which can hold your shiny gold bag directly, plus some other bags.
A dark orange bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
A light red bag, which can hold bright white and muted yellow bags, either of which could then hold your shiny gold bag.
So, in this example, the number of bag colors that can eventually contain at least one shiny gold bag is 4.

How many bag colors can eventually contain at least one shiny gold bag? (The list of rules is quite long; make sure you get all of it.)

In [ ]:
import re
bags = list(map(str, open('day07_example2.txt', 'r').read().split('\n')))

In [ ]:
bags[0:3]

['shiny gold bags contain 2 dark red bags.',
 'dark red bags contain 2 dark orange bags.',
 'dark orange bags contain 2 dark yellow bags.']

In [ ]:
re.findall('([\w ]*) bags contain ', bags[0])

['shiny gold']

In [ ]:
re.findall('(\d+) ([\w ]*) bag(?:s)?', bags[0])

[('2', 'dark red')]

In [ ]:
# Let's set up an empty dictionary for the rules

set_of_rules = {}

# now let's go through the rules. Note that each bag is going to contain multiple bags, so we'll have nested dictionaries. ugh.

for bag in bags:
    outer_bag = re.findall('([\w ]*) bags contain ', bag)[0] # color of the outer bag
    set_of_rules[outer_bag] = {}
    inner_bags = re.findall('(\d+) ([\w ]*) bag(?:s)?', bag) #numbers and colors of inner bags. we'll add these to the dictionary we created above.
    for bag in inner_bags:
        set_of_rules[outer_bag][bag[1]] = int(bag[0])


In [ ]:
set_of_rules

{'shiny gold': {'dark red': 2},
 'dark red': {'dark orange': 2},
 'dark orange': {'dark yellow': 2},
 'dark yellow': {'dark green': 2},
 'dark green': {'dark blue': 2},
 'dark blue': {'dark violet': 2},
 'dark violet': {}}

In [ ]:
# Here, we've got to make a function that checks the outer bag and sees whether it contains our bag of interest, 'shiny gold' in it.
# If it does, return True. Additionally, we need to keep track of which bags we have already checked for shiny gold. I'm thinking
# we can create an empty list in our next function called path, look at all the inner bags, if we haven't already checked it already 
# go to this bag, check it for shiny gold.

def going_down_the_path(outer_bag,path):
    if 'shiny gold' in set_of_rules[outer_bag]: #Eheck the inner dictionary for shiny gold. If it's here, we've found it!
        return True
    else:
        for inner_bag in set_of_rules[outer_bag]: 
            if inner_bag not in path: # This means we haven't checked this bag yet.
                path.append(inner_bag)
                if going_down_the_path(inner_bag,path):
                    return True
        return False # This bag doesn't contain shiny gold.

In [ ]:
# All we have to do for part 1 now is set up a counter at 0, and the empty list called path. I probably could have just done this with the previous function,
# but maybe this is best I did it this way, not knowing what is coming up in part 2.

def traverse_part1():
    count = 0
    for outer_bag in set_of_rules:
        path = []
        if going_down_the_path(outer_bag,path):
            count +=1
    print(f'The number of bag colors that can eventually contain at least one shiny gold bag is {count}')

In [ ]:
traverse_part1()

The number of bag colors that can eventually contain at least one shiny gold bag is 0


## Part Two 
It's getting pretty expensive to fly these days - not because of ticket prices, but because of the ridiculous number of bags you need to buy!

Consider again your shiny gold bag and the rules from the above example:

```
faded blue bags contain 0 other bags.
dotted black bags contain 0 other bags.
vibrant plum bags contain 11 other bags: 5 faded blue bags and 6 dotted black bags.
dark olive bags contain 7 other bags: 3 faded blue bags and 4 dotted black bags.
```

So, a single shiny gold bag must contain 1 dark olive bag (and the 7 bags within it) plus 2 vibrant plum bags (and the 11 bags within each of those):
 1 + 1*7 + 2 + 2*11 = 32 bags!

Of course, the actual rules have a small chance of going several levels deeper than this example; be sure to count all of the bags, 
even if the nesting becomes topologically impractical!

Here's another example:

```
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
```
In this example, a single shiny gold bag must contain 126 other bags.

How many individual bags are required inside your single shiny gold bag?

In [ ]:
# More recursion here. We're starting with the shiny gold bag as our outside bag. We'll begin with a counter of 0. The trivial case is if there are no bags in the
# left. Otherwise, we'll go to each inner bag, and count how many of them there are and add to the count. Next we'll want to run this function again on the inner bag
# and multiply this number by the amount of bags we found. 

def count_bags(outer_bag):
    count = 0
    if set_of_rules[outer_bag] is not bool:
        return 0
    else:
        # Go to each bag inside of the shiny gold bag
        for inner_bag in set_of_rules[outer_bag]:
            count += set_of_rules[outer_bag][inner_bag]
            count += set_of_rules[outer_bag][inner_bag] * count_bags(inner_bag)
        return count



In [ ]:
count_bags('shiny gold')

0